In [1]:
import os
import sys

sys.path.append("..")

from itertools import product
from lightgbm import LGBMRanker
from sklearn.utils import check_random_state
from xai_ranking.benchmarks import (
    human_in_the_loop_experiment,
    hierarchical_ranking_explanation,
    lime_experiment,
    shap_experiment,
    sharp_experiment,
    # rank_lime_experiment,
    # participation_experiment,
)
from xai_ranking.preprocessing import (
    preprocess_atp_data,
    preprocess_csrank_data,
    preprocess_higher_education_data,
    preprocess_movers_data,
    preprocess_synthetic_data,
)
from xai_ranking.datasets import (
    fetch_atp_data,
    fetch_csrank_data,
    fetch_higher_education_data,
    fetch_movers_data,
    fetch_synthetic_data,
)
from xai_ranking.scorers import (
    atp_score,
    csrank_score,
    higher_education_score,
    synthetic_equal_score_3ftrs,
)
from xai_ranking.metrics import (
    # explanation_sensitivity,
    explanation_sensitivity_all_neighbors,
    outcome_sensitivity,
    bootstrapped_explanation_consistency,
    cross_method_explanation_consistency,
    cross_method_outcome_consistency,
    outcome_fidelity,
)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kendalltau
from mlresearch.utils import check_random_states, set_matplotlib_style
from xai_ranking.utils import scores_to_ordering
import seaborn as sns

RNG_SEED = 42

In [2]:
# # Set up ranker for the moving company dataset:
# X, ranks, score = preprocess_movers_data(fetch_movers_data(test=False))
# qids_train = X.index.value_counts().to_numpy()

# model = LGBMRanker(
#     objective="lambdarank", label_gain=list(range(max(ranks) + 1)), verbose=-1
# )
# model.fit(
#     X=X,
#     y=ranks,
#     group=qids_train,
# )

In [3]:
datasets = [
    {
        "name": "ATP",
        "data": preprocess_atp_data(fetch_atp_data()),
        "scorer": atp_score,
    },
    {
        "name": "CSRank",
        "data": preprocess_csrank_data(fetch_csrank_data()),
        "scorer": csrank_score,
    },
    {
        "name": "Higher Education",
        "data": preprocess_higher_education_data(
            fetch_higher_education_data(year=2020)
        ),
        "scorer": higher_education_score,
    },
    # {
    #     "name": "Moving Company",
    #     "data": preprocess_movers_data(fetch_movers_data(test=True)),
    #     "scorer": model.predict,
    # },
    {
        "name": "Synthetic_0",
        "data": preprocess_synthetic_data(
            fetch_synthetic_data(synth_dt_version=0, item_num=2000)
        ),
        "scorer": synthetic_equal_score_3ftrs,
    },
    {
        "name": "Synthetic_1",
        "data": preprocess_synthetic_data(
            fetch_synthetic_data(synth_dt_version=1, item_num=2000)
        ),
        "scorer": synthetic_equal_score_3ftrs,
    },
    {
        "name": "Synthetic_2",
        "data": preprocess_synthetic_data(
            fetch_synthetic_data(synth_dt_version=2, item_num=2000)
        ),
        "scorer": synthetic_equal_score_3ftrs,
    },
]
xai_methods = [
    {
        "iterations": 1,
        "name": "LIME",
        "experiment": lime_experiment,
        "type": "score",
        "kwargs": {"mode": "regression"},  # classification, regression
    },
    {
        "iterations": 1,
        "name": "SHAP",
        "experiment": shap_experiment,
        "type": "score",
        "kwargs": {},
    },
    {
        "iterations": 1,
        "name": "ShaRP_RANK",
        "experiment": sharp_experiment,
        "type": "rank",
        "kwargs": {
            "qoi": "rank",
            "verbose": True,
            "sample_size": None,
            "measure": "shapley",
            "n_jobs": -1,
            "replace": False,
        },
    },
    {
        "iterations": 1,
        "name": "ShaRP_SCORE",
        "experiment": sharp_experiment,
        "type": "score",
        "kwargs": {
            "qoi": "rank_score",
            "verbose": True,
            "sample_size": None,
            "measure": "shapley",
            "n_jobs": -1,
            "replace": False,
        },
    },
    # {
    #     "iterations": 1,
    #     "name": "ShaRP_TOPK",
    #     "experiment": sharp_experiment,
    #     "kwargs": {
    #         "qoi": "top_k",
    #         "verbose": True,
    #         "sample_size": None,
    #         "measure": "shapley",
    #         "n_jobs": -1,
    #         "replace": True,
    #     },
    # },
    {
        "iterations": 1,
        "name": "HRE_DT",
        "type": "score",
        "experiment": hierarchical_ranking_explanation,
        "kwargs": {"model_type": "DT", "s": 10},  # DT, LR, OLS, PLS
    },
    {
        "iterations": 1,
        "name": "HRE_LR",
        "type": "score",
        "experiment": hierarchical_ranking_explanation,
        "kwargs": {"model_type": "LR", "s": 10},  # DT, LR, OLS, PLS
    },
    {
        "iterations": 1,
        "name": "HRE_OLS",
        "type": "score",
        "experiment": hierarchical_ranking_explanation,
        "kwargs": {"model_type": "OLS", "s": 10},  # DT, LR, OLS, PLS
    },
    {
        "iterations": 1,
        "name": "HRE_PLS",
        "type": "score",
        "experiment": hierarchical_ranking_explanation,
        "kwargs": {"model_type": "PLS", "s": 10},  # DT, LR, OLS, PLS
    },
    # {
    #     "iterations": 1,
    #     "name": "HIL_Shapley",
    #     "experiment": human_in_the_loop_experiment,
    #     "kwargs": {"upper_bound": 1, "lower_bound": None, "method_type": "shapley"},
    # },
    {
        "iterations": 1,
        "name": "HIL_Standardized-Shapley",
        "type": "std-score",
        "experiment": human_in_the_loop_experiment,
        "kwargs": {
            "upper_bound": 1,
            "lower_bound": None,
            "method_type": "standardized shapley",
        },
    },
    {
        "iterations": 1,
        "name": "HIL_Rank-Shapley",
        "experiment": human_in_the_loop_experiment,
        "type": "std-rank",
        "kwargs": {
            "upper_bound": 1,
            "lower_bound": None,
            "method_type": "rank-relevance shapley",
        },
    },
    # {
    #     "iterations": 1,
    #     "name": "RankLIME",
    #     "experiment": rank_lime_experiment,
    #     "kwargs": {
    #         "explanation_size": 3,
    #         "rank_similarity_coefficient": lambda x, y: kendalltau(x, y)[0],
    #         "individual_masking": True,
    #         "use_entry": 0,
    #         "use_pandas_where": False,
    #     },
    # },
    # {"iterations": 1, "name": "Participation", "experiment": participation_experiment},
]

total_states = sum(map(lambda x: x["iterations"], xai_methods)) * len(datasets)
random_states = (x for x in check_random_states(RNG_SEED, total_states))

In [4]:
# # Uncomment to run full experiment

# results = {}
# for dataset in datasets:
#     results[dataset["name"]] = {}
#     for xai_method in xai_methods:
#         results[dataset["name"]][xai_method["name"]] = []

#         experiment_func = xai_method["experiment"]
#         score_func = dataset["scorer"]

#         X, ranks, scores = dataset["data"]

#         for iteration_idx in range(xai_method["iterations"]):
#             random_state = next(random_states)
#             result_fname = (
#                 f"results/contributions/_contributions_{dataset['name']}_"
#                 f"{xai_method['name']}_{iteration_idx}.csv"
#             )

#             if os.path.exists(result_fname):
#                 print(f"{result_fname} exists, skipping")
#                 continue
#             if (
#                 xai_method["name"] in ("HRE_LR", "HRE_PLS")
#                 and dataset["name"] == "Moving Company"
#             ):
#                 # dataset has binary categorical data
#                 # specified methods cannot handle such data
#                 continue
#             if (
#                 xai_method["name"].startswith("HIL")
#                 and dataset["name"] == "Moving Company"
#             ):
#                 # method requires weights, that are absent fot L2R
#                 continue

#             kwargs = {} if "kwargs" not in xai_method else xai_method["kwargs"]
#             if dataset["name"] == "Moving Company" and xai_method["name"].startswith(
#                 "ShaRP"
#             ):
#                 kwargs["sample_size"] = 150

#             contributions = experiment_func(
#                 X, score_func, random_state=random_state, **kwargs
#             )

#             results[dataset["name"]][xai_method["name"]].append(contributions)
#             result_df = pd.DataFrame(contributions, columns=X.columns, index=X.index)
#             result_df.to_csv(result_fname)

In [5]:
def read_results_from_files():
    return_dict = {}
    for cur_dataset in datasets:
        return_dict[cur_dataset["name"]] = {}
        for cur_xai_method in xai_methods:
            return_dict[cur_dataset["name"]][cur_xai_method["name"]] = []
            for iteration in range(cur_xai_method["iterations"]):
                fname = (
                    f"results/contributions/_contributions_"
                    f"{cur_dataset['name']}_{cur_xai_method['name']}_"
                    f"{iteration}.csv"
                )
                if os.path.isfile(fname):
                    (
                        return_dict[cur_dataset["name"]][cur_xai_method["name"]].append(
                            pd.read_csv(fname, index_col=0)
                        )
                    )
    return return_dict

In [7]:
results = read_results_from_files()

In [10]:
# results

# Sensitivity Plots

In [ ]:
methods = [method for method in results["ATP"].keys() if not method.startswith("BATCH")]

for method in methods:
    for dataset in datasets:
        rankings = scores_to_ordering(dataset["scorer"](dataset["data"][0]))
        try:

            result = explanation_sensitivity_all_neighbors(
                dataset["data"][0], 
                results[dataset["name"]][method][0],
                rankings,
                measure="euclidean",
                normalization=False,
                #n_features=0.8,
                threshold=0.1
            )
    
            df_length = dataset['data'][0].shape[0]
            for plot_idx in [int(num/10*df_length) for num in range(1,10,1)]:
                measure_distances, rank_distances, feature_distances = result(row_idx=plot_idx)
        
                temp = pd.DataFrame()
        
                temp['Explanation distance'] = measure_distances
                temp['Absolute Rank distance'] = np.absolute(rank_distances)
                temp['Feature distance'] = feature_distances
                sns.scatterplot(data=temp, x="Explanation distance", y="Absolute Rank distance", hue="Feature distance")
                leg = plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
                # plt.title(f'{method} and {dataset["name"]} and rank={plot_idx}')
                # sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
        
                plt.savefig(
                f"results/sensitivity-scatterplot-{dataset["name"]}-{method}-{plot_idx}.pdf",
                format="pdf",
                bbox_inches="tight",
                transparent=True
                )
                plt.show()
                # plt.clf()
        except Exception:
            pass

# Explanation Sensitivity

In [35]:
# methods = [method for method in results["ATP"].keys() if not method.startswith("BATCH")]

for method in xai_methods:
    # Set up Euclidean normalizer
    # X = dataset["data"][0]
    # # Get scores
    # scorer = dataset["scorer"]
    # scores = np.array(scorer(dataset["data"][0]))
    # # Set contributions
    # res = results[dataset["name"]][method["name"]][0]
    # if method["type"] == "rank":
    #     max_target = X.shape[0]
    # elif method["type"] == "score":
    #     max_target = scores.max()
    # elif method["type"] == "std-score":
    #     res = results[dataset["name"]][method["name"]][0].div(result.sum(axis=1), axis=0)
    #     max_target = scores.max()
    # else:
    #     max_target = None
    #     print("Unknown QoI")

    # print(method["name"], max_target)
    
    # for dataset in datasets:
    #     rankings = scores_to_ordering(dataset["scorer"](dataset["data"][0]))
    #     # try:
    #     result = explanation_sensitivity_all_neighbors(
    #         dataset["data"][0], 
    #         results[dataset["name"]][method["name"]][0],
    #         rankings,
    #         measure="euclidean",
    #         threshold=0.1,
    #         normalizer=max_target
    #     )

    #     print(result)

    #     df_length = dataset['data'][0].shape[0]
    #     for plot_idx in [int(num/10*df_length) for num in range(1,10,1)]:
    #         measure_distances, rank_distances, feature_distances = result(row_idx=plot_idx)
    
    #         temp = pd.DataFrame()
    
    #         temp['Explanation distance'] = measure_distances
    #         temp['Absolute Rank distance'] = np.absolute(rank_distances)
    #         temp['Feature distance'] = feature_distances
    #         sns.scatterplot(data=temp, x="Explanation distance", y="Absolute Rank distance", hue="Feature distance")
    #         leg = plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    #         # plt.title(f'{method} and {dataset["name"]} and rank={plot_idx}')
    #         # sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    
    #         plt.savefig(
    #         f"results/sensitivity-scatterplot-{dataset["name"]}-{method["name"]}-{plot_idx}.pdf",
    #         format="pdf",
    #         bbox_inches="tight",
    #         transparent=True
    #         )
    #         plt.show()
    #         # plt.clf()
    #     # except Exception:
    #     #     print(Exception)
    #     #     pass
    for dataset in datasets:
        print(method["name"])
        print("\t",dataset["name"])
        print("\t\t",results[dataset["name"]][method["name"]][0].sum(axis=1).max())

LIME
	 ATP
		 36.604759844719894
LIME
	 CSRank
		 4.615001550675019
LIME
	 Higher Education
		 40.835405768333125
LIME
	 Synthetic_0
		 0.2678895302454555
LIME
	 Synthetic_1
		 0.3024799849414474
LIME
	 Synthetic_2
		 0.2501936708961683
SHAP
	 ATP
		 44.65465116279049
SHAP
	 CSRank
		 15.862186567174469
SHAP
	 Higher Education
		 57.913524999999815
SHAP
	 Synthetic_0
		 0.2812447409084132
SHAP
	 Synthetic_1
		 0.22316847395411019
SHAP
	 Synthetic_2
		 0.22875032273320559
ShaRP_RANK
	 ATP
		 42.46511627906977
ShaRP_RANK
	 CSRank
		 93.95767195767195
ShaRP_RANK
	 Higher Education
		 697.9856836077307
ShaRP_RANK
	 Synthetic_0
		 999.5
ShaRP_RANK
	 Synthetic_1
		 999.5
ShaRP_RANK
	 Synthetic_2
		 999.5
ShaRP_SCORE
	 ATP
		 44.65465116279069
ShaRP_SCORE
	 CSRank
		 15.64175135859853
ShaRP_SCORE
	 Higher Education
		 60.63824087329995
ShaRP_SCORE
	 Synthetic_0
		 0.3026047623368028
ShaRP_SCORE
	 Synthetic_1
		 0.20665922776877269
ShaRP_SCORE
	 Synthetic_2
		 0.2553306180317151
HRE_DT
	 ATP
	

# Outcome Sensitivity

In [ ]:
help(outcome_sensitivity)

In [ ]:
methods = [method for method in results["ATP"].keys() if not method.startswith("BATCH")]

out_sens_res = {}
out_sens_sem = {}
for method in methods:
    out_sens_res[method] = {}
    out_sens_sem[method] = {}
    for dataset in datasets:
        rankings = scores_to_ordering(dataset["scorer"](dataset["data"][0]))
        result = outcome_sensitivity(
            dataset["data"][0],
            dataset["scorer"],
            results[dataset["name"]][method][0],
            threshold=0.8,
            n_neighbors=10,
            n_tests=10,
            std_multiplier=0.2,
            aggregate_results=True,
            random_state=RNG_SEED,
        )
        out_sens_res[method][dataset["name"]] = result[0]
        out_sens_sem[method][dataset["name"]] = result[1]

In [ ]:
pd.DataFrame(out_sens_res)

# Explanation Consistency

In [ ]:
help(cross_method_explanation_consistency)

In [ ]:
methods = [method for method in results["ATP"].keys() if not method.startswith("BATCH")]

exp_cons_res = {}
exp_cons_sem = {}
for dataset in datasets:
    exp_cons_res[dataset["name"]] = pd.DataFrame(index=methods, columns=methods)
    exp_cons_sem[dataset["name"]] = pd.DataFrame(index=methods, columns=methods)
    for method1, method2 in product(methods, methods):
        result = cross_method_explanation_consistency(
            results[dataset["name"]][method1][0],
            results[dataset["name"]][method2][0],
            measure="jaccard",
            n_features=2,
        )

        exp_cons_res[dataset["name"]].loc[method1, method2] = result[0]
        exp_cons_sem[dataset["name"]].loc[method1, method2] = result[1]

In [ ]:
avg_exp_cons_res = pd.DataFrame(columns=methods, index=methods).fillna(0)
for dataset, res_ in exp_cons_res.items():
    avg_exp_cons_res += res_

avg_exp_cons_res /= len(datasets)
avg_exp_cons_res

In [ ]:
exp_cons_res["CSRank"]

# Outcome Consistency

In [ ]:
help(cross_method_outcome_consistency)

In [ ]:
methods = [method for method in results["ATP"].keys() if not method.startswith("BATCH")]

out_cons_res = {}
out_cons_sem = {}
for dataset in datasets:
    out_cons_res[dataset["name"]] = pd.DataFrame(index=methods, columns=methods)
    out_cons_sem[dataset["name"]] = pd.DataFrame(index=methods, columns=methods)
    for method1, method2 in product(methods, methods):
        result = cross_method_outcome_consistency(
            dataset["data"][0],
            dataset["scorer"],
            results[dataset["name"]][method1][0],
            results[dataset["name"]][method2][0],
            random_state=RNG_SEED,
        )

        out_cons_res[dataset["name"]].loc[method1, method2] = result[0]
        out_cons_sem[dataset["name"]].loc[method1, method2] = result[1]

In [ ]:
out_cons_res["Higher Education"]

avg_out_cons_res = pd.DataFrame(columns=methods, index=methods).fillna(0)
for dataset, res_ in out_cons_res.items():
    if dataset != "Moving Company":
        avg_out_cons_res += res_

avg_out_cons_res /= len(datasets)
avg_out_cons_res

# Bootstrapped Consistency

In [ ]:
help(bootstrapped_explanation_consistency)

In [ ]:
methods = [method for method in results["ATP"].keys() if not method.startswith("BATCH")]

boot_cons_res = {}
boot_cons_sem = {}
for method in methods:
    boot_cons_res[method] = {}
    boot_cons_sem[method] = {}
    for dataset in datasets:
        try:
            result = bootstrapped_explanation_consistency(
                results[dataset["name"]][method][0],
                results[dataset["name"]][f"BATCH_{method}"],
                measure="euclidean",
            )
            boot_cons_res[method][dataset["name"]] = result[0]
            boot_cons_sem[method][dataset["name"]] = result[1]
        except:
            pass

# Fidelity

In [8]:
outcome_fidelity

<function xai_ranking.metrics._fidelity.outcome_fidelity(contributions, target, avg_target, target_max=1, target_pairs=None, rank=True)>

In [9]:
results["ATP"]

{'LIME': [                         serve__pct_1st_serve  \
  player_name                                     
  John Isner                           5.683096   
  Nick Kyrgios                         5.764958   
  Reilly Opelka                        1.982074   
  Hubert Hurkacz                      -1.208558   
  Matteo Berrettini                    2.079879   
  ...                                       ...   
  Diego Schwartzman                    5.898979   
  Fabio Fognini                       -6.081595   
  Cristian Garin                      -1.189482   
  Bernabe Zapata Miralles              1.549742   
  Nikoloz Basilashvili                -6.276985   
  
                           serve__pct_1st_serve_points_won  \
  player_name                                                
  John Isner                                      8.418830   
  Nick Kyrgios                                    8.005311   
  Reilly Opelka                                   8.320181   
  Hubert Hurkacz

In [10]:
dataset["scorer"]

NameError: name 'dataset' is not defined

In [ ]:
# FOR RANKING QOI
methods = [method for method in results["ATP"].keys() if not method.startswith("BATCH")]
dataset_names = [dataset["name"] for dataset in datasets]

fidelity_res = pd.DataFrame(index=dataset_names, columns=methods)
for dataset in datasets:
    for method in methods:
        try:
            target = scores_to_ordering(dataset["scorer"](dataset["data"][0]))
            result = outcome_fidelity(
                results[dataset["name"]][method][0],
                target,
                target.mean(),
                target_max=target.size,
                rank=True,
            )
            fidelity_res.loc[dataset["name"], method] = result
        except:
            pass

fidelity_res

In [ ]:
# FOR SCORE QOI
methods = [method for method in results["ATP"].keys() if not method.startswith("BATCH")]
dataset_names = [dataset["name"] for dataset in datasets]

fidelity_res = pd.DataFrame(index=dataset_names, columns=methods)
for dataset in datasets:
    for method in methods:
        try:
            target = dataset["scorer"](dataset["data"][0])
            result = outcome_fidelity(
                results[dataset["name"]][method][0],
                target,
                target.mean(),
                target_max=target.max(),
                rank=False,
            )
            fidelity_res.loc[dataset["name"], method] = result
        except:
            pass

fidelity_res

In [ ]:
datasets[2]["data"][0]

In [ ]:
datasets[3]["name"]

In [ ]:
datasets[3]["data"][0]

# Old metrics

In [ ]:
from xai_ranking.metrics.old_metrics import (
    compute_all_agreement,
    compute_all_fidelity,
    compute_all_sensitivity,
    compute_all_stability,
)

In [ ]:
def plot_dataset_aggregated_summary(agg_mean, agg_sem, gap=0.3):
    n_cols = len(agg_mean.columns)
    x = np.arange(n_cols).astype(np.float64)

    methods = agg_mean.index.unique()
    bars = len(methods)
    bar_width = (1 - gap) / bars
    x -= (bars - 1) * bar_width / 2

    for method in methods:
        plt.errorbar(
            x,
            agg_mean.loc[method],
            yerr=agg_sem.loc[method],
            marker="o",
            label=method,
            linestyle="None",
        )
        x += bar_width
    plt.legend()
    plt.xticks(np.arange(n_cols), agg_mean.columns, rotation=45)

In [ ]:
set_matplotlib_style(font_size=12, **{"font.family": ["Nimbus Roman"]})
aggregated_summary, aggregated_error = compute_all_stability(results, axis=0)
for dataset in aggregated_summary:
    plot_dataset_aggregated_summary(
        aggregated_summary[dataset], aggregated_error[dataset]
    )
    plt.title(dataset)
    plt.show()

# Agreement

In [ ]:
agreement_results = compute_all_agreement(results, n_features=3)
agreement_results["ATP"]["kendall"]

In [ ]:
agreement_results["ATP"]["jaccard"]

In [ ]:
agreement_results["CSRank"]["kendall"]

In [ ]:
agreement_results["CSRank"]["jaccard"]

In [ ]:
agreement_results["Higher Education"]["kendall"]

# Sensitivity

In [ ]:
sensitivity_results = compute_all_sensitivity(
    original_data=datasets, results=results, n_neighbors=10
)

In [ ]:
pd.DataFrame(sensitivity_results[0])

# Fidelity

In [ ]:
fidelity_results = compute_all_fidelity(
    original_data=datasets, results=results, random_state=RNG_SEED
)
pd.DataFrame(fidelity_results[0])

In [ ]:
pd.DataFrame(fidelity_results[0]).mean(1)  # .drop(columns="Moving Company").mean(1)